In [ ]:
#######################################################
#                LIBRARIES AND MODULES                #
#######################################################
import pandas
import pprint
import gffutils
import re
from   Bio                    import SeqIO
from   Bio.Seq                import Seq, MutableSeq, reverse_complement, transcribe, back_transcribe, translate
from   Bio.SeqFeature         import SeqFeature, SimpleLocation
from   Bio.SeqUtils.ProtParam import ProteinAnalysis
from   BCBio                  import GFF
from   BCBio.GFF              import GFFExaminer

#######################################################
#                     INPUT FILES                     #
#######################################################

bb31_gff = '../raw/blast/bbB31.gff'
bb31_fna = '../raw/blast/bbB31.fna'

#pangen_ref = '../raw/pan_genome_reference.fa' # I will eventually add in the whole blast report generation step and ID_to_group.tsv generation step. maybe that's worthy of its own script?
blast_results = '../raw/blast/pan_genome_blast.topHit.tsv'
seqID_to_group = '../raw/blast/ID2GROUP.tsv'

#######################################################
#                     OUTPUT FILES                    #
#######################################################

#ID_to_group  = "../raw/ID_to_group.tsv"
gwas_output_table = "../raw/groupID_to_BBgene.tsv"
wetlab_output_table = "../raw/groupID_to_BBgene_wetlab.tsv"
all_genes_table = "../raw/all_genes_peptide_info.tsv"

#######################################################
#               PLASMID TRANSLATION TABLE             #
#######################################################

plasmid_name_table={"NC_001318.1":"chromosome",
                    "NC_000957.1":"lp5",
                    "NC_001904.1":"cp9",
                    "NC_001849.2":"lp17",
                    "NC_000955.2":"lp21",
                    "NC_001850.1":"lp25",
                    "NC_001903.1":"cp26",
                    "NC_001851.2":"lp28-1",
                    "NC_001852.1":"lp28-2",
                    "NC_001853.1":"lp28-3",
                    "NC_001854.1":"lp28-4",
                    "NC_000948.1":"cp32-1",
                    "NC_000949.1":"cp32-3",
                    "NC_000950.1":"cp32-4",
                    "NC_000951.1":"cp32-6",
                    "NC_000952.1":"cp32-7",
                    "NC_000953.1":"cp32-8",
                    "NC_000954.1":"cp32-9",
                    "NC_001855.1":"lp36",
                    "NC_001856.1":"lp38",
                    "NC_001857.2":"lp54",
                    "NC_000956.1":"lp56"}


In [ ]:
#######################################################
#                     PROCESS HITS                    #
#######################################################

#Fields="query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score"
results = pandas.read_csv(blast_results, sep = '\t', header=None) # read in blast results .tsv

#create ID to Group translation table
#{{TODO}}: ID2GROUP.tsv is created by sed on the pan-genome reference fasta. I need to rewrite my sed script in python to plug into this workflow.

id2group = pandas.read_csv(seqID_to_group, sep = '\t', header=None)
id2group = pandas.DataFrame({"ID":id2group[0],"group":id2group[1]})

#create a dataframe containing group, geneID, %identity, alignment length, and E-score
hits = pandas.DataFrame({"ID":results[0],
                         "gene":results[1],
                         "percent_ident":results[2],
                         "alignment_length": results[3],
                         "E-score": results[10]})

hits = hits.drop_duplicates() # I have no idea why blast output duplicates for each hit.... drop them
ids = list(hits['ID'].unique()) # pull out unique ID numbers to filter the list by.
hits = hits.loc[(hits["ID"].isin(ids)) & (hits["gene"].str.startswith("gene-BB"))] # filter list by IDs then pull out all BB-RS genes

#The below two transforms re-order the table into alphabetical order so it makes visual checks much more tricky. prob need to write something to check this. the numbers in -> out match.
grouped = hits.groupby('ID') # group rows by ID
first_hits = grouped.first() # pull the first row from each ID's group
translated_hits = first_hits.merge(id2group,how='left', on = 'ID') #merge em together

# this was to check for the presence of a specific gene that was being omitted from the final results. It was present but not being parsed
# for whatever reason. This can be removed.
#translated_hits[translated_hits.isin(['gene-BB_RS05835'])].stack()

In [3]:
#######################################################
#      Get GFF summary for genes and pseudogenes      #
#######################################################

examiner = GFFExaminer()
in_handle = open(bb31_gff)
gff_stats = examiner.available_limits(in_handle) # get high level overview of all features in GFF
in_handle.close()

num_gene = gff_stats['gff_type']['gene',] # set number of genes to summary value from above
num_pseudo = gff_stats['gff_type']['pseudogene',] # set number of pseudogenes to summary value from above
num_cds= gff_stats['gff_type']['CDS',]
num_features = num_gene + num_pseudo # get total number of features. this is to confirm correct parsing later on.
print("         number of genes: ", num_gene)
print("   number of pseudogenes: ", num_pseudo)
print(" number of annotated CDS: ", num_cds)
print("total number of features: ", num_features)


         number of genes:  1486
   number of pseudogenes:  68
 number of annotated CDS:  1515
total number of features:  1554


In [4]:
#######################################################
#                      PARSE GFF                      #
#######################################################

# parse B31 genome
records = SeqIO.to_dict(SeqIO.parse(bb31_fna, "fasta"))
prot_records = SeqIO.to_dict(SeqIO.parse(bb31_fna, "fasta"))

limit_info = dict(gff_type=["CDS"])

# parse B31 GFF for gene_id parsing, this could probably be done with the protein stuff but BCBio's GFF parser does not parse both for whatever reason. I can either parse genes OR CDS but attempting to do both only gets me genes and I dont feel like re-writing my dict comprehension at this time.
in_handle = open(bb31_gff) #set up input handle
for rec in GFF.parse(in_handle):
    records[rec.id].features = rec.features

# parse B31 GFF for protein stuff
in_handle = open(bb31_gff) #set up input handle
for rec in GFF.parse(in_handle, limit_info=limit_info):
    prot_records[rec.id].features = rec.features



In [ ]:
#######################################################
#              Peptide/Protein Analysis               #
#######################################################

i = 0 # init counting variable
protein_info = {} # init dict for protein info.
plasmid_ID = "" # init plasmid ID

for plasmid in prot_records.keys(): # start dict comprehension, go through each plasmid
    current_plasmid = prot_records[plasmid] # set current_plasmid as the current iteration through the records dictionary, each key is an individual plasmid
    plasmid_ID = plasmid_name_table[current_plasmid.id] # translate NCBI accession ID to the common identifier

    for gene in current_plasmid.features: # within each plasmid, look at the SeqFeatures and then iterate through each one.
        if gene.id.startswith("NC_"): # if gene.id starts with NC_, it is a larger structure and not a gene, ignore it. the previous method of filtering is not optimal at all and leaves out genes randomly for whatever reason. This method is functional.
            pass # skip this feature and iterate loop

        else: # if it does not start with the chromosome ID, then pull out the below values
            if gene.type == 'CDS':
                #print(gene.type) # sanity check and debugging
                current_feature = gene # set current_feature to the current iteration of dict comprehension. THIS IS REQUIRED
                locus_tag = current_feature.qualifiers['locus_tag'] # extract the current gene locus ID (this is the same as the gene-BB_RS ID used elsewhere and output by blast.)
                locus_tag = "gene-"+str(locus_tag).strip('\[\]\'') # gotta add the gene- to make it play ball

                protein_info[locus_tag] = {} # add current gene to keys in dict, init nested dict.
                AA_length = 0 # init length counter
                AA_incomplete = False # init boolean for sequence completeness
                N_count = 0 # init N counter
                AA_ambiguous = False # init boolean for codon ambiguity
                Ambig_count = 0 # init ambiguous counter
                raw_seq = str(gene.extract(current_plasmid.seq)) # this works, attempted to use a MutableSeq earlier but I'm not working within the biopython class aside from the initial parsing so it isn't needed/definitely will break.

                while len(raw_seq) % 3 != 0: # check for multiple of 3 via modulo, if remainder is not 0 then loop over the following
                    AA_incomplete = True # set boolean to true
                    raw_seq = raw_seq+"N" # append an N to the end
                    #print(gene.id,"added an N") # sanity check
                    N_count += 1 # iterate N counter.

                AA_seq_raw = translate(raw_seq, table=11, to_stop=True) # translate raw_seq to peptide seq based on translation table 11.

                if re.search("[XJBZ]", AA_seq_raw) is not None: # check for ambiguous AAs
                    AA_ambiguous = True # set boolean to true
                    Ambig_count = len(re.findall("[XJBZ]", AA_seq_raw))

                #####
                AA_Seq = AA_seq_raw.replace("*","").replace("X","").replace("J","L").replace("B","N").replace("Z","Q") # this is probably wrong, change ambiguous AAs to the non-acid form and flat out remove the fully ambiguous AAs (X) so the calculations will work.
                # perhaps I should be generating all possible peptides based on ambiguity and then feeding each into the below calculations. Need to think about/get advice on.
                #####
                AA_length = len(AA_Seq)
                peptide = ProteinAnalysis(str(AA_Seq)) # set up biopython protein analysis object to use the below called methods.

                # !! I continually get a div by zero error when it should not be giving me that. I am manually counting AAs and running the percent function to see where it is going wrong.
                # !! update it was dragging in ncRNA and that did not play nicely for whatever reason. fixed.
                # !! OK so the div/0 error arises when non CDS peptide translations are fed into the counting method. It really despises ncRNA and refuses to count AAs for whatever reason. It was fixed with upstream filtering.

                #print(AA_incomplete, AA_ambiguous) # Am I breaking this peptide sequence early on? is the ambiguity breaking it? is the substitution breaking it? give me info.
                #AA_content = peptide.count_amino_acids() # count em
                #print(AA_content) # display the count
                #AA_percent = peptide.get_amino_acids_percent() # this is where the div/0 error arises because the above count fails to count.
                #print(AA_percent) # if this works, the rest will too.

                # lets get some values for wetlab now.
                AA_mol_weight        = peptide.molecular_weight() # get molecular weight
                AA_aromaticity       = peptide.aromaticity() # calculate aromaticity based on AA composition
                AA_isoelectric       = peptide.isoelectric_point() # calculate isoelectric point
                AA_instability_index = peptide.instability_index() # calculate instability index
                AA_gravy             = peptide.gravy() # calculate overall hydropathy. {{TODO}} perhaps this would be better represented by individual hydropathy plots for each protein.
                AA_sec_struct        = peptide.secondary_structure_fraction() # estimatate percent composition of secondary structures.
                AA_helix             = AA_sec_struct[0] # extract percent_helix
                AA_turn              = AA_sec_struct[1] # extract percent_turn
                AA_sheet             = AA_sec_struct[2] # extract percent_beta_sheet

                protein_info[locus_tag] = {'plasmid': plasmid_ID,
                                             'gene' : locus_tag,
                                        'protein_id': gene.id,
                                  'bool_incomplete' : AA_incomplete,
                                         'added_Ns' : N_count,
                            'bool_ambiguous_codons' : AA_ambiguous,
                                  'ambiguous_count' : Ambig_count,
                                        'AA_length' : AA_length,
                                 'molecular_weight' : AA_mol_weight,
                                      'aromaticity' : AA_aromaticity,
                                'isoelectric_point' : AA_isoelectric,
                                'instability_index' : AA_instability_index,
                                            'GRAVY' : AA_gravy,
                                       'perc_helix' : AA_helix,
                                        'perc_turn' : AA_turn,
                                       'perc_sheet' : AA_sheet,
                                         'gene_seq' : raw_seq,
                                      'peptide_seq' : AA_Seq } # save to dict

                # The below would be nice in a log file maybe. This data needs its own table for all genes in BB31
                ###################################################################################
                #print("      Gene Locus ID: ", locus_tag)
                #print("         protein ID: ", gene.id)
                #print("                Seq: ", AA_Seq) # translate current CDS to AA via table 11.
                #print("Incomplete peptide?: ", AA_incomplete)
                #print(" Number of Ns added: ", N_count)
                #print("     Ambiguous AAs?: ", AA_ambiguous)
                #print(" Ambiguous AA Count: ", Ambig_count)
                #print("     number of AA's: ",len(AA_Seq))
                #print("   molecular weight: ","%0.2f" % AA_mol_weight)
                #print("        aromaticity: ","%0.2f" % AA_aromaticity)
                #print("  isoelectric point: ","%0.2f" % AA_isoelectric)
                #print("  instability index: ","%0.2f" % AA_instability_index)
                #print("              GRAVY: ","%0.2f" % AA_gravy)
                #print("           helix \%: ","%0.2f" % AA_helix)
                #print("            turn \%: ","%0.2f" % AA_turn)
                #print("           sheet \%: ","%0.2f" % AA_sheet)
                #print(i) # print current iteration
                i += 1 # iterate counter
                ###################################################################################

            else: # if not CDS, skip this feature.
                #print("not CDS, skipping")
                pass

protein_info = pandas.DataFrame.from_dict(protein_info, orient='index') # turn the dict into a pandas.DataFrame
#pprint.pprint(protein_info) # sanity check, look at the table output

In [ ]:
#######################################################
#                 Translate ID to Gene                #
#######################################################

# {{TODO}}
# I need to write some generalized functions to pull this info from GFF files since this is a simple dictionary comprehension
# function that I have used previously, although with a few subtle changes.
# a generalized function would be very helpful to basically set the desired outputs and a dict to store them in.
# or something like that. idk, I'll deal with that after this becomes functional.
# actually this might be built into BCBio.GFF somewhere. Need to look into that.

ID_to_locus_tag = {} # set up empty dict for IDs and Tags
for plasmid in records.keys(): # start dict comprehension, go through each plasmid
    current_plasmid = records[plasmid] # set current_plasmid as the current iteration through the records dictionary, each key is an individual plasmid
    for gene in current_plasmid.features: # within each plasmid, look at the SeqFeatures and then iterate through each one.
        current_feature = gene # set current_feature to the current iteration of dict comprehension. THIS IS REQUIRED
        #print("current_plasmid: ", current_plasmid.name) # sanity check
        #print("current feature: ") # sanity check
        #print(current_feature) # sanity check
        if current_feature.id.startswith("NC_"): # the previous method of filtering is not optimal at all and leaves out genes randomly for whatever reason. This method is functional. {{TODO}}: NEEDS WORK
            pass # skip this feature and iterate loop
        else: # if it does not start with the chromosome ID, then pull out the below values
            ID = str(current_feature.id) # this is the ID value that is also present in the blast hits table.
            ID = str(ID.strip('\[\]\'')) # make it format nicely so downstream stuff works correctly
            #locus_tag = str(current_feature.qualifiers.get('locus_tag')) # this is the above ID value but without gene- at the beginning. This is redundant and not really necessary atm
            #locus_tag = locus_tag.strip('\[\]\'') # see above
            old_locus_tag = str(current_feature.qualifiers.get('old_locus_tag', 'NA')) # not all features have an old locus tag, if not present, fill with NA
            old_locus_tag = old_locus_tag.strip('\[\]\'') # see double above
            ID_to_locus_tag[ID] = old_locus_tag

print(len(ID_to_locus_tag)) # check length and compare with # of genes present in GFF
if len(ID_to_locus_tag) == num_features:
    print("features in parsed table is equal to total number of features in GFF. It appears to have parsed successfully.")
else:
    print("number of features in parsed table is not equal to the number of features present in the GFF. Something went wrong!")


# the below is for visual confirmation of successful parsing. There is no doubt a way to do this more efficiently
#i = 1
#for key in ID_to_locus_tag:
#    print(key, ID_to_locus_tag[key])
#    print(i)
#    i+=1

1554
features in parsed table is equal to total number of features in GFF. It appears to have parsed successfully.


In [ ]:
#######################################################
#                  ANNOTATION PARSING                 #
#######################################################

# lets try to get product info using gffutils
# gff parsing using gffutils is below since BCBio.GFF only extracts parent features......

db = gffutils.create_db(bb31_gff, dbfn = 'bb31.db', force=True, gtf_subfeature='CDS', merge_strategy="create_unique")
db = gffutils.FeatureDB('bb31.db')
#
gene_annotations = {} # init dict for annotations
for key in ID_to_locus_tag: # iterate through each ID
    for f in db.children(key): # for each gene present in the GFF database created above,
        gene = str(f['Parent']).strip('\[\]\'') # extract out the gene ID and strip the surrounding mess.
        product = f['product'] # extract gene product

        if 'Ontology_term' in f.attributes: # check to see if GO term is present
            ont_term = str(f['Ontology_term']) # extract term as string
        else: # if no term set NA
            ont_term = 'NA'

        if 'go_function' in f.attributes: # check for GO function
            go_fxn = str(f['go_function']) # extract function as str
        else: # otherwise NA
            go_fxn = 'NA'

        if 'go_process' in f.attributes: # check for GO process
            go_proc = str(f['go_process']) # extract process as str
        else: # otherwise NA
            go_proc = 'NA'

        gene_annotations[key] = {'gene' : gene,'Product':product, 'GO_term':ont_term, 'GO_process':go_proc, 'GO_function' : go_fxn} # populate annotation dict

gene_annotations = pandas.DataFrame.from_dict(gene_annotations, orient='index') # convert to pandas.DataFrame


In [8]:
#######################################################
#                   MERGE DATAFRAMES                  #
#######################################################

# create column in hits df and populate with results of dictionary lookup for each gene present
translated_hits['locus_tag'] = translated_hits['gene'].map(ID_to_locus_tag) # map genes to old locus tags
# merge annotations table with translated hits table.
translated_hits = translated_hits.merge(gene_annotations, how='left', on='gene') # merge annotations
translated_hits = translated_hits.merge(protein_info, how='left', on='gene') # merge peptide info
translated_hits['gene'] = translated_hits['gene'].str.removeprefix("gene-") # strip this annoying prefix
translated_hits['protein_id'] = translated_hits['protein_id'].str.removeprefix("cds-") # and this one too

## Also add locus tag to the protein info table
protein_table = protein_info # no touch original table
protein_table['locus_tag'] = protein_info['gene'].map(ID_to_locus_tag) # map gene ID to locus tag
protein_table = protein_table.merge(gene_annotations, how='left', on='gene') # merge with gene annotations
protein_table['gene'] = protein_table['gene'].str.removeprefix("gene-") # strip annoying prefix
protein_table['protein_id'] = protein_table['protein_id'].str.removeprefix("cds-") # strip annoying prefix

In [ ]:
#######################################################
#                   REORDER COLUMNS                   #
#######################################################

#print(*enumerate(translated_hits.columns))

# INITIAL ORDERING
# (0, 'ID') (1, 'gene') (2, 'percent_ident') (3, 'alignment_length') (4, 'E-score') (5, 'group') (6, 'locus_tag') (7, 'Product')
# (8, 'GO_term') (9, 'GO_process') (10, 'GO_function') (11, 'plasmid') (12, 'protein_id') (13, 'bool_incomplete') (14, 'added_Ns')
# (15, 'bool_ambiguous_codons') (16, 'ambiguous_count') (17, 'AA_length') (18, 'molecular_weight') (19, 'aromaticity') (20, 'isoelectric_point')
# (21, 'instability_index') (22, 'GRAVY') (23, 'perc_helix') (24, 'perc_turn') (25, 'perc_sheet') (26, 'gene_seq') (27, 'peptide_seq')

updated_table = translated_hits.iloc[:,[0,5,11,1,6,2,3,4,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]] # reorder columns
#print(updated_table) # sanity check
table_no_blast = translated_hits.iloc[:,[0,5,1,6,11,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]] # remove blast stuff

#print(*enumerate(updated_table.columns))

# and work with the whole genome protein table
#print(*enumerate(protein_table.columns))
all_proteins = protein_table.iloc[:,[0,1,18,2,19,7,8,3,4,5,6,9,10,11,12,13,14,15,20,21,22,16,17]]
#print(*enumerate(all_genes_table.columns))

In [10]:
#######################################################
#                  WRITE OUTPUT TABLE                 #
#######################################################
all_proteins.to_csv(all_genes_table, sep='\t', header=True, index=False) # output all of the above generated peptide information to its own tsv file :)
updated_table.to_csv(gwas_output_table, sep='\t', header=True, index=None) # write table with blast data for GWAS team
table_no_blast.to_csv(wetlab_output_table, sep='\t', header= True, index=None) # write table without blast data for wetlab team

In [ ]:
input_pangenome_reference =